# Utils

Conté el codi per descarregar i carregar el model GPT-2 dels repositoris oficials d'OpenAI. A posteriori carregarem els seus pesos, tokenitzador i hiperparàmetre.

In [23]:
import json
import os
import re

import numpy as np
import requests
import tensorflow as tf
from tqdm import tqdm

#Importem el encoder que tenim dins de la carpeta de Modules. (Encoder BPE d'OpenAI GPT-2.)
from modules.encoder import get_encoder

## download_gpt2_files

Funció per descarregar els fitxers de GPT2.

In [24]:
def download_gpt2_files(model_size, model_dir): #Model_dir = Path complert ex: "models/124M"
    assert model_size in ["124M", "355M", "774M", "1558M"]

    tf_ckpt_path = tf.train.latest_checkpoint(model_dir)

    if tf_ckpt_path:  
        return
    
    os.makedirs(model_dir, exist_ok=True)
    url = "https://openaipublic.blob.core.windows.net/gpt-2/models"
    for filename in [
        "checkpoint",
        "encoder.json",
        "hparams.json",
        "model.ckpt.data-00000-of-00001",
        "model.ckpt.index",
        "model.ckpt.meta",
        "vocab.bpe",
    ]:
        r = requests.get(f"{url}/{model_size}/{filename}", stream=True)
        r.raise_for_status()

        with open(os.path.join(model_dir, filename), "wb") as f:
            file_size = int(r.headers["content-length"])
            chunk_size = 1000
            with tqdm(
                ncols=100,
                desc="Fetching " + filename,
                total=file_size,
                unit_scale=True,
                unit="b",
            ) as pbar:
                # 1k for chunk_size, since Ethernet packet size is around 1500 bytes
                for chunk in r.iter_content(chunk_size=chunk_size):
                    f.write(chunk)
                    pbar.update(chunk_size)

In [25]:
#Ens baixem tots els models en les seves respectives carpetes amb aquesta execució, fent una 
model_sizes = ["124M", "355M", "774M", "1558M"]
for model_size in model_sizes:
    download_gpt2_files(model_size=model_size, model_dir="models/"+model_size)

##### Explicació de la funció 

In [26]:
#Aquesta mida de models és la que ens proporciona OpenAI si no controlem això no ens podrem baixar els fitxers ja que la URL serà incorrecte.
#Definim la variable del directori del model.
model_dir= "models"
model_size= "124M"
model_full_path= os.path.join(model_dir, model_size)

In [27]:
#Amb l'assert ens assegurem que la mida que li passem de model està dins de la llista que definim. Si posem un model size que no està dins de la llista que definim obtindrem "AssertionError"
assert model_size in model_sizes

In [28]:
#Amb tensorflow anirem a buscar l'últim checkpoint en la ruta del model. 
tf_ckpt_path = tf.train.latest_checkpoint(model_full_path)

In [29]:
#Retornarà la ruta del model si existeix i res si no existeix.
#Si per algo el model s'ha baixat incomplet, ho detectariem.
tf_ckpt_path

'models\\124M\\model.ckpt'

In [30]:
#Si existeix el model, no ens el baixem perquè si no per cada prompt estariem sobreescrivint el model.
if tf_ckpt_path:  
    pass #Return

#Si no existeix el model, ens creem la carpeta del model i a posteriori ens el baixem.
os.makedirs(model_full_path, exist_ok=True)

In [31]:
#Tenim els noms de fitxers del Model per baixar-nos emmagatzemats en una variable. Són els que anirem a atacar en la URL.
filenames = ["checkpoint",
        "encoder.json",
        "hparams.json",
        "model.ckpt.data-00000-of-00001",
        "model.ckpt.index",
        "model.ckpt.meta",
        "vocab.bpe"]

In [32]:
#Creo la carpeta del Directori aqui manualment perquè lo següent funcioni
#Ens definim la URL del Azure Blob Storage Públic d'OpenAI.   
url = "https://openaipublic.blob.core.windows.net/gpt-2/models"
 
#Si existeix el model, no ens el baixem perquè si no per cada prompt estariem sobreescrivint el model.
if tf_ckpt_path:  
    pass

#Repeteixo codi en aquesta part perquè sinó sempre al executar el notebook sencer se'ns posarà a baixar el model. Això ho controlem en la funció.
else:
    #Si no existeix el model, ens creem la carpeta del model i a posteriori ens el baixem.
    os.makedirs(model_full_path, exist_ok=True)
    #Per cada fitxer del model a descarregar 
    for filename in filenames:
        #Atacarem la URL del Blob Públic amb el recurs en qüestió gràcies a la llibreria requests fent una petició HTTP.
        #Stream=True --> Si no es fa servir aquest paràmetre al fer la petició del request tot el contingut de la resposta es descarregarà inmediatament a la memòria abans de processar-lo. Això pot provocar un s elevat de memòria i causar problemes de rendiment si no es disposa de suficient memòria per manejar la càrrega completa del fitxer.
        #Al fer servir el Stream permet processar el contingut per parts (chunks), reduïnt l'ús de memòria millorant el maneig de grans volums de dades
        r = requests.get(f"{url}/{model_size}/{filename}", stream=True)
        #Comprovem que la petició s'ha fet correctament. Si no fa res la resposta és exitosa (codi 200) de lo contrari llença una excepcióo (per ex 400 o 500)
        r.raise_for_status() # --> No retorna res si està OK però és un objecte del tipus: <bound method Response.raise_for_status of <Response [200]>>
        # Creem carpeta, serà "models", i l'arxiu que ens descarreguem (estem creant la ruta completa). Obrirem en mode escritura binaria ja que els arxius que ens baixarem son binaris (com els pesos dels models)
        with open(os.path.join(model_full_path, filename), "wb") as f:
                #Obtindrem la mida total de l'arxiu a descarregar gràcies als encapçalats de la resposta http. Ens servirà per mostrar amb el TQDM el progrés de la descàrrega.
                file_size = int(r.headers["content-length"])
                #Definim la mida de cada chunk o bloc que llegirem i ens baixarem de l'arxiu en cada iteració, s'estableix a 1000 bytes.
                #Si la mida del fitxer és de 10GB i definim el chunksize en 1000, dividim la resposta en 10 parts.
                chunk_size = 1000
                #Amb el TQDM ens permet mostrar el progrés de la descàrrega amb una barra de progrés.
                with tqdm(
                    ncols=100, #La amplada de la barra de progrés que es mostrarà en el Output.
                    desc="Fetching " + filename, #Descripció en la barra de progrés.
                    total=file_size, #El número total d'iteracions esperades, será el mateix que la mida del fitxer.
                    unit_scale=True, #Més format de la barra progressiva.
                    unit="b", #La mida de la barra de progrés es escala amb la mida de l'arxiu que volem baixar.
                ) as pbar:
                    #Gràcies al strem=true del requests permetem que el contingut es descarregui en parts en lloc de carregar tot el contingut de cop en la memòria.
                    #Llavors el r.iter_content genera parts de contingut descarregat depenent de la mida especificada pel chunk_size
                    for chunk in r.iter_content(chunk_size=chunk_size):
                        #Escribim cada part de contingut del chunk al fitxer.
                        f.write(chunk)
                        #Actualitzem la barra de progrés amb la mida del chunk que hem descarregat.
                        pbar.update(chunk_size)
                    


## def load_gpt2_params_from_tf_ckpt(tf_ckpt_path, hparams):

In [33]:
#Funció per carregar els paràmetres del model des d'un arxiu Checkpoint de Tensorflow.

def load_gpt2_params_from_tf_ckpt(tf_ckpt_path, hparams):  #Passem el path del checkpoint i els hiperparàmetres.
    def set_in_nested_dict(d, keys, val):
        if not keys:
            return val
        if keys[0] not in d:
            d[keys[0]] = {}
        d[keys[0]] = set_in_nested_dict(d[keys[0]], keys[1:], val)
        return d

    params = {"blocks": [{} for _ in range(hparams["n_layer"])]}
    for name, _ in tf.train.list_variables(tf_ckpt_path):
        array = np.squeeze(tf.train.load_variable(tf_ckpt_path, name))
        name = name[len("model/") :]
        if name.startswith("h"):
            m = re.match(r"h([0-9]+)/(.*)", name)
            n = int(m[1])
            sub_name = m[2]
            set_in_nested_dict(params["blocks"][n], sub_name.split("/"), array)
        else:
            set_in_nested_dict(params, name.split("/"), array)

    return params

In [34]:
def set_in_nested_dict(d, keys, val):
        if not keys:
            return val
        if keys[0] not in d:
            d[keys[0]] = {}
        d[keys[0]] = set_in_nested_dict(d[keys[0]], keys[1:], val)
        return d

In [35]:
d, keys, val = {}, [], []


In [36]:
if not keys:
    print(val)
    pass
if keys[0] not in d:
    d[keys[0]] = {}
d[keys[0]] = set_in_nested_dict(d[keys[0]], keys[1:], val)
print(d)



[]


IndexError: list index out of range

## def load_encoder_hparams_and_params(model_size, models_dir):

In [38]:
def load_encoder_hparams_and_params(model_size, models_dir):
    assert model_size in ["124M", "355M", "774M", "1558M"]

    model_dir = os.path.join(models_dir, model_size)
    download_gpt2_files(model_size, model_dir)
    tf_ckpt_path = tf.train.latest_checkpoint(model_dir)

    encoder = get_encoder(model_size, models_dir)
    hparams = json.load(open(os.path.join(model_dir, "hparams.json")))
    params = load_gpt2_params_from_tf_ckpt(tf_ckpt_path, hparams)

    return encoder, hparams, params

In [39]:
model = load_encoder_hparams_and_params(model_size="124M", models_dir="models") 

##### Explicació de la funció 

In [40]:
#PARAM 1 --> La mida del model
model_size = "124M"

#PARAM 2 --> Directori del model
models_dir = "models"

In [41]:
#Verificarem que el model que ens arriba per paràmetre està dins de la llista dels models permesos. 
#Amb l'assert si no es cumpleix es llença una excepció AssertionError.
assert model_size in ["124M", "355M", "774M", "1558M"]

In [42]:
#Ens montem el path complet del nostre model: per ex models/124M 
model_dir = os.path.join(models_dir, model_size)

In [43]:
#Cridem a la funció de descàrrega dels fitxers del model.
#Si el model en qüestió no el tenim baixat o el tenim a mitges ens el baixarà, de lo contrari no farà res.
download_gpt2_files(model_size, model_dir)

In [44]:
#Obtenim el darrer checkpoint del model.
tf_ckpt_path = tf.train.latest_checkpoint(model_dir)
tf_ckpt_path

'models\\124M\\model.ckpt'

In [51]:
#Ens carreguem el encoder que aquesta funció carrega el encoder.json i el vocab.bpe.
encoder = get_encoder(model_size, models_dir)
encoder

In [52]:
vars(encoder)

{'encoder': {'!': 0,
  '"': 1,
  '#': 2,
  '$': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  '(': 7,
  ')': 8,
  '*': 9,
  '+': 10,
  ',': 11,
  '-': 12,
  '.': 13,
  '/': 14,
  '0': 15,
  '1': 16,
  '2': 17,
  '3': 18,
  '4': 19,
  '5': 20,
  '6': 21,
  '7': 22,
  '8': 23,
  '9': 24,
  ':': 25,
  ';': 26,
  '<': 27,
  '=': 28,
  '>': 29,
  '?': 30,
  '@': 31,
  'A': 32,
  'B': 33,
  'C': 34,
  'D': 35,
  'E': 36,
  'F': 37,
  'G': 38,
  'H': 39,
  'I': 40,
  'J': 41,
  'K': 42,
  'L': 43,
  'M': 44,
  'N': 45,
  'O': 46,
  'P': 47,
  'Q': 48,
  'R': 49,
  'S': 50,
  'T': 51,
  'U': 52,
  'V': 53,
  'W': 54,
  'X': 55,
  'Y': 56,
  'Z': 57,
  '[': 58,
  '\\': 59,
  ']': 60,
  '^': 61,
  '_': 62,
  '`': 63,
  'a': 64,
  'b': 65,
  'c': 66,
  'd': 67,
  'e': 68,
  'f': 69,
  'g': 70,
  'h': 71,
  'i': 72,
  'j': 73,
  'k': 74,
  'l': 75,
  'm': 76,
  'n': 77,
  'o': 78,
  'p': 79,
  'q': 80,
  'r': 81,
  's': 82,
  't': 83,
  'u': 84,
  'v': 85,
  'w': 86,
  'x': 87,
  'y': 88,
  'z': 89,
  '{': 9

In [65]:
#Ens carreguem el JSON dels hiperparámetres.
#n_vpcab: aquest model té en el seu vocabulari/moodel 50257 tokens (paraules o subparaules)
hparams = json.load(open(os.path.join(model_dir, "hparams.json")))
hparams



{'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 768, 'n_head': 12, 'n_layer': 12}

In [72]:
import gc
#Depenent de quin model ens carreguem, canvia el n_embd, els n_head i els n_layer. Però tots estan entrenats amb el mateix vocabulari.
#Aqui veurem les diferències en els hiperparàmetres de cadascun d'elles.
for model_size in model_sizes:
    _model = load_encoder_hparams_and_params(model_size, "models")
    print(f'+++ HIPERPARAMETERS MODEL SIZE:{model_size} --> {_model[1]} \n')

    #Alliberem la memòria per obrir el següent model, sino es va acumulant i consumeix més recursos.
    del _model
    #Garbage Collector, administra de forma automàtica la memòria i allibera els objectes que ja no estan en ús.
    gc.collect()

+++ HIPERPARAMETERS MODEL SIZE:124M --> {'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 768, 'n_head': 12, 'n_layer': 12} 

+++ HIPERPARAMETERS MODEL SIZE:355M --> {'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 1024, 'n_head': 16, 'n_layer': 24} 

+++ HIPERPARAMETERS MODEL SIZE:774M --> {'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 1280, 'n_head': 20, 'n_layer': 36} 

+++ HIPERPARAMETERS MODEL SIZE:1558M --> {'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 1600, 'n_head': 25, 'n_layer': 48} 

